In [240]:
import os
import yaml

from google.oauth2 import service_account
import os
import yaml
import git

## source vars

path = os.path.expanduser('~')

profile_pass = os.path.join(path,".droughty/profile.yaml")

with open(profile_pass) as f:
    lookml_config = yaml.load(f, Loader=yaml.FullLoader)

def get_git_root(path):

        git_repo = git.Repo(path, search_parent_directories=True)
        git_root = git_repo.git.rev_parse("--show-toplevel")
        return (git_root)
    
    
git_def_path = get_git_root(os.getcwd())

git_path = git_def_path

filename = 'droughty_project.yaml'

droughty_project = os.path.join(git_path,filename)

with open(droughty_project) as f:
    enviroment_project = yaml.load(f, Loader=yaml.FullLoader)



for key,value in enviroment_project.items():
    
    if key == 'profile':

        if lookml_config[value]['warehouse_name'] == 'big_query':

            if value in lookml_config:

        ## global vars 

                warehouse_name =  lookml_config[value]['warehouse_name']
                project_name =  lookml_config[value]['project_name']
                schema_name =  lookml_config[value]['schema_name']
                test_schemas = lookml_config[value]['test_schemas']
                
                warehouse_schema =   """

                with source as (

                    select * from `{0}.{1}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`

                    )

                    select * from source

                """.format(project_name,schema_name)

                dbml_reference_dict = """


                with source as (

                select * from `{0}.{1}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`

                ),

                    pks as (
                    select 
                    table_name as pk_table_name,
                    column_name as pk_column_name,
                    trim(column_name, "_pk") as pk_sk,
                    from source
                    where column_name like '%pk%'
                    ),

                    fks as (
                    select
                    table_name as fk_table_name,
                    column_name as fk_column_name,
                    trim(column_name, "_fk") as fk_sk,
                    from source
                    where column_name like '%fk%'

                    ),

                    references as (
                    select * from pks

                    inner join fks on pks.pk_sk = fks.fk_sk

                    )

                    select 
                    
                    *except (pk_column_name,pk_table_name),

                    case when pk_column_name is null
                        then 'not_available'
                    else pk_column_name
                    end as pk_column_name,

                    case when pk_table_name is null
                        then 'not_available'
                    else pk_table_name
                    end as pk_table_name                   
                    from source

                    left join references on source.column_name = references.fk_column_name and references.fk_table_name = source.table_name
                
                """.format(project_name,schema_name)

        elif lookml_config[value]['warehouse_name'] == 'snowflake':
            
            if value in lookml_config:

                warehouse_name =  lookml_config[value]['warehouse_name']
                project_name =  lookml_config[value]['project_name']
                schema_name =  lookml_config[value]['schema_name']
                test_schemas = lookml_config[value]['test_schemas']
                database = lookml_config[value]['database']

                snowflake_schema = '''

                select * from {0}.information_schema.columns;

                '''.format(database)

    ## warehouse test schemas

test_warehouse_schema =   """

        with source_1 as (

            select * from `{0}.{1}.INFORMATION_SCHEMA.COLUMNS`

            ),

        source_2 as (

        select * from `{0}.{2}.INFORMATION_SCHEMA.COLUMNS`
        
        ),
        
        source_3 as (

        select * from `{0}.{3}.INFORMATION_SCHEMA.COLUMNS`
        
        ),
        
        unioned as (

        select * from source_1
        
        union all
        
        select * from source_2
        
        union all
        
        select * from source_3
        
        )

        select * from unioned

""".format(project_name,test_schemas[0],test_schemas[1],test_schemas[2])

explores = (enviroment_project.get("explores"))


explore_tables = []

for key,value in explores.items():

    for key,value in value.items():

        explore_tables.append(value)
        
single_list_tables = [i[0] for i in explore_tables]

        
lookml_explore_schema =     """


                        with source as (

                        select * from `{0}.{1}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`

                        where table_name in ('{2}','{3}','{4}')

                        ),
                

                        pks as (
                            select 
                            table_name as pk_table_name,
                            column_name as pk_column_name,
                            trim(column_name, "_pk") as pk_sk,
                            from source
                            where column_name like '%pk%'
                            ),

                        fks as (
                            select
                            table_name as fk_table_name,
                            column_name as fk_column_name,
                            trim(column_name, "_fk") as fk_sk,
                            from source
                            where column_name like '%fk%'

                            ),


                            select 

                            pk_table_name,
                            pk_column_name,
                            fk_table_name,
                            pk_table_name as pk_table_name_value,
                            fk_column_name
                            from pks

                            inner join fks on pks.pk_sk = fks.fk_sk



        """.format(project_name,schema_name,single_list_tables[0],single_list_tables[1],single_list_tables[2])

In [340]:
user_transaction_template = '''

with source as (

select * from `{{project_id}}.{{schema_id}}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
where table_name in 
{% for tables in table_names [2:] %} {{ table_names | inclause }} {% endfor %}

),

{% for value in table_names  %}

explore_table_row_count_{{ value | sqlsafe }} as (

select 

'{{ value | sqlsafe }}' as table_name,
count(*) as {{ value | sqlsafe }}_row_count

from `{{project_id}}.{{schema_id}}.{{ value | sqlsafe }}`

group by 1

),

{% endfor %}


merge_counts as (

select * from source 

{% for value in table_names  %}

left join 

explore_table_row_count_{{value | sqlsafe }} on source.table_name = explore_table_row_count_{{value | sqlsafe }}.table_name

{% endfor %}

),

pks as (
    select 
    table_name as pk_table_name,
    column_name as pk_column_name,
    trim(column_name, "_pk") as pk_sk,
    from source
    where column_name like '%pk%'
),

fks as (
    select
    table_name as fk_table_name,
    column_name as fk_column_name,
    trim(column_name, "_fk") as fk_sk,
    from source
    where column_name like '%fk%'

),
    
final as (

select 

pk_table_name,
pk_column_name,
fk_table_name,
fk_column_name,


from pks

inner join fks on pks.pk_sk = fks.fk_sk


select * from 

select * from merge_counts



'''

In [341]:
params

{'project_id': 'ra-development',
 'schema_id': 'jordan_analytics_dev',
 'table_names': ['wh_marketing_website_event_pages_fact',
  'wh_marketing_website_users_dim',
  'wh_marketing_website_event_tracks_fact']}

In [342]:
params = {
    'project_id': project_name,
    'schema_id': schema_name, 
    'table_names': flat_list
    
}

explores = (enviroment_project.get("explores"))


In [343]:
explores

{'sales_explore': {'parent_table': ['wh_marketing_website_event_pages_fact'],
  'dimensions': ['wh_marketing_website_users_dim'],
  'facts': ['wh_marketing_website_event_tracks_fact']}}

In [344]:
params

{'project_id': 'ra-development',
 'schema_id': 'jordan_analytics_dev',
 'table_names': ['wh_marketing_website_event_pages_fact',
  'wh_marketing_website_users_dim',
  'wh_marketing_website_event_tracks_fact']}

In [345]:
flat_list = []
for sublist in explore_tables:
    for item in sublist:
        flat_list.append(item)

In [346]:
flat_list

['wh_marketing_website_event_pages_fact',
 'wh_marketing_website_users_dim',
 'wh_marketing_website_event_tracks_fact']

In [347]:
from jinjasql import JinjaSql
j = JinjaSql(param_style='pyformat')
query, bind_params = j.prepare_query(user_transaction_template,params)

In [348]:
print(query)



with source as (

select * from `%(project_id_1)s.%(schema_id_2)s.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
where table_name in 
 (%(inclause_3)s,%(inclause_4)s,%(inclause_5)s) 

),



explore_table_row_count_wh_marketing_website_event_pages_fact as (

select 

'wh_marketing_website_event_pages_fact' as table_name,
count(*) as wh_marketing_website_event_pages_fact_row_count

from `%(project_id_6)s.%(schema_id_7)s.wh_marketing_website_event_pages_fact`

group by 1

),



explore_table_row_count_wh_marketing_website_users_dim as (

select 

'wh_marketing_website_users_dim' as table_name,
count(*) as wh_marketing_website_users_dim_row_count

from `%(project_id_8)s.%(schema_id_9)s.wh_marketing_website_users_dim`

group by 1

),



explore_table_row_count_wh_marketing_website_event_tracks_fact as (

select 

'wh_marketing_website_event_tracks_fact' as table_name,
count(*) as wh_marketing_website_event_tracks_fact_row_count

from `%(project_id_10)s.%(schema_id_11)s.wh_marketing_website_event_

In [349]:
print(query % bind_params)



with source as (

select * from `ra-development.jordan_analytics_dev.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
where table_name in 
 (wh_marketing_website_event_pages_fact,wh_marketing_website_users_dim,wh_marketing_website_event_tracks_fact) 

),



explore_table_row_count_wh_marketing_website_event_pages_fact as (

select 

'wh_marketing_website_event_pages_fact' as table_name,
count(*) as wh_marketing_website_event_pages_fact_row_count

from `ra-development.jordan_analytics_dev.wh_marketing_website_event_pages_fact`

group by 1

),



explore_table_row_count_wh_marketing_website_users_dim as (

select 

'wh_marketing_website_users_dim' as table_name,
count(*) as wh_marketing_website_users_dim_row_count

from `ra-development.jordan_analytics_dev.wh_marketing_website_users_dim`

group by 1

),



explore_table_row_count_wh_marketing_website_event_tracks_fact as (

select 

'wh_marketing_website_event_tracks_fact' as table_name,
count(*) as wh_marketing_website_event_tracks_fact_row_cou

In [350]:
from six import string_types
isinstance(value, string_types)

False

In [52]:
from six import string_types
def quote_sql_string(value):
    '''
    If `value` is a string type, escapes single quotes in the string
    and returns the string enclosed in single quotes.
    '''
    if isinstance(value, string_types):
        new_value = str(value)
        new_value = new_value.replace("'", "''")
        return "'{}'".format(new_value)
    return value

In [53]:
from copy import deepcopy
def get_sql_from_template(query, bind_params):
    if not bind_params:
        return query
    params = deepcopy(bind_params)
    for key, val in params.items():
        params[key] = quote_sql_string(val)
    return query % params

In [54]:
from jinjasql import JinjaSql

def apply_sql_template(template, parameters):
    '''
    Apply a JinjaSql template (string) substituting parameters (dict) and return
    the final SQL.
    '''
    j = JinjaSql(param_style='pyformat')
    query, bind_params = j.prepare_query(template, parameters)
    return get_sql_from_template(query, bind_params)

In [55]:
sql = get_sql_from_template(query, bind_params)

In [56]:
sql

"\n\nwith source as (\n\nselect * from `'ra-development'.'jordan_analytics_dev'.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`\nwhere table_name in (\n, (['wh_website_event_pages_fact'],['wh_website_users_dim'],['wh_website_event_tracks_fact', 'wh_website_test_fact'])\n\n),\n\n\n\nexplore_table_row_count_['wh_website_event_tracks_fact', 'wh_website_test_fact'] as (\n\nselect \n\ntable_name,\ncount(*)\n\nfrom source \n\nwhere table_name = ['wh_website_event_pages_fact']\n\ngroup by 1\n\n\n\nexplore_table_row_count_['wh_website_event_tracks_fact', 'wh_website_test_fact'] as (\n\nselect \n\ntable_name,\ncount(*)\n\nfrom source \n\nwhere table_name = ['wh_website_event_pages_fact']\n\ngroup by 1\n\n\n\nexplore_table_row_count_['wh_website_event_tracks_fact', 'wh_website_test_fact'] as (\n\nselect \n\ntable_name,\ncount(*)\n\nfrom source \n\nwhere table_name = ['wh_website_event_pages_fact']\n\ngroup by 1\n\n\n\n"

In [336]:
import os
def strip_blank_lines(text):
    '''
    Removes blank lines from the text, including those containing only spaces.
    https://stackoverflow.com/questions/1140958/whats-a-quick-one-liner-to-remove-empty-lines-from-a-python-string
    '''
    return os.linesep.join([s for s in text.splitlines() if s.strip()])

In [40]:
enviroment_project

{'profile': 'ra-development',
 'explores': {'sales_explore': {'parent_table': ['wh_website_event_pages_fact'],
   'dimensions': ['wh_website_users_dim'],
   'facts': ['wh_website_event_tracks_fact', 'wh_website_test_fact']}}}

In [41]:
explore_tables = []

for key,value in explores.items():

    for key,value in value.items():

        explore_tables.append(value)

In [42]:
explore_tables

[['wh_website_event_pages_fact'],
 ['wh_website_users_dim'],
 ['wh_website_event_tracks_fact', 'wh_website_test_fact']]

In [106]:
single_list_tables = [i for i in explore_tables]


In [103]:
single_list_tables = [i [0] for i in explore_tables]


In [97]:
single_list_tables

['wh_website_event_pages_fact',
 'wh_website_users_dim',
 'wh_website_event_tracks_fact']

In [88]:
single_list_tables = [i [0:2] for i in explore_tables]


In [112]:
single_list_tables

[['wh_website_event_pages_fact'],
 ['wh_website_users_dim'],
 ['wh_website_event_tracks_fact', 'wh_website_test_fact']]

In [110]:
single_list_tables_test = [i [0] for i in single_list_tables]

In [111]:
single_list_tables_test

['wh_website_event_pages_fact',
 'wh_website_users_dim',
 'wh_website_event_tracks_fact']

In [113]:
explore_tables

[['wh_website_event_pages_fact'],
 ['wh_website_users_dim'],
 ['wh_website_event_tracks_fact', 'wh_website_test_fact']]

In [114]:
type(explore_tables)

list

In [115]:
flattened = [] 
for sublist in explore_tables: 
    for val in sublist: 
        flattened.append(val) 

X_inter.iloc[:, flattened]

NameError: name 'X_inter' is not defined

In [119]:
explore_tables[0:4]

[['wh_website_event_pages_fact'],
 ['wh_website_users_dim'],
 ['wh_website_event_tracks_fact', 'wh_website_test_fact']]

In [122]:
explore_tables = []

for key,value in explores.items():

    for key,value in value.items():

        explore_tables.append(value)

In [124]:
explore_tables

[['wh_website_event_pages_fact'],
 ['wh_website_users_dim'],
 ['wh_website_event_tracks_fact', 'wh_website_test_fact']]